In [ ]:

# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Site Selection in Las Vegas using Places Insights and BigQuery**

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/nevada_site_selection/places_insights_nevada_site_selection_example.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgooglemaps-samples%2Finsights-samples%2Fmain%2Fplaces_insights%2Fnotebooks%2Fnevada_site_selection%2Fplaces_insights_nevada_site_selection_example.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/nevada_site_selection/places_insights_nevada_site_selection_example.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/nevada_site_selection/places_insights_nevada_site_selection_example.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

**Overall Goal**

This notebook demonstrates a multi-stage site selection workflow for a new coffee shop in Las Vegas. It combines broad competitive analysis, custom commercial suitability scoring, and target market density analysis to identify prime locations, then visualizes the results on a combined, interactive map.

**Key Technologies Used**

*  **[Places Insights](https://developers.google.com/maps/documentation/placesinsights)**: To provide the core Places dataset and the `PLACES_COUNT_PER_H3` function.
*   **[BigQuery](https://cloud.google.com/bigquery):**: To perform large-scale geospatial analysis and calculate suitability scores.
*   **[Google Maps Place Details API](https://developers.google.com/maps/documentation/places/web-service/place-details):** To fetch rich, detailed information (name, address, rating) for specific ground-truth locations.
*   **[Google Maps 2D Tiles](https://developers.google.com/maps/documentation/tile/2d-tiles-overview):** To use Google Maps as the interactive basemap.
*   **Python Libraries:**
    * **[GeoPandas](https://geopandas.org/en/stable/)** for spatial data manipulation.
    * **[Folium](https://python-visualization.github.io/folium/latest/)** for creating the final interactive, layered map.

See [Google Maps Platform Pricing](https://mapsplatform.google.com/intl/en_uk/pricing/) For API costs assocated with running this notebook.

**The Step-by-Step Workflow**

1.  **Analyze Competitor Density:** We begin by using BigQuery to analyze the distribution of major competitor brands across Clark County ZIP codes. This initial step helps identify broad areas with lower market saturation.

2.  **Identify Prime Commercial Zones:** The notebook then runs a more sophisticated query to calculate a custom suitability score for H3 hexagonal cells. This score is based on the weighted density of complementary businesses (restaurants, bars, casinos, tourist attractions), pinpointing the most commercially vibrant areas.

3.  **Find Target Market Hotspots & Synthesize:** Next, we use the `PLACES_COUNT_PER_H3` function to find the density of our target business type—coffee shops. The notebook then **automatically** cross-references these coffee shop counts with the highest-scoring suitability zones to identify the most promising cells for a new location.

4.  **Create a Combined Visualization:** In the final step, we generate a single, layered map. The **base layer** is a choropleth "heatmap" showing the suitability scores across Las Vegas. The **top layer** displays individual pins for existing coffee shops in the top-ranked zones, providing a direct, ground-level view of the current market landscape.

**How to Use This Notebook**

1.  **\*\*Set Up Secrets:\*\*** Before you begin, you must configure two secrets in the Colab “Secrets” tab (the 🔑 key icon on the left menu):
    *   `GCP_PROJECT`: Your Google Cloud Project ID with access to Places Insights.
    *   `GMP_API_KEY`: Your Google Maps Platform API key. Ensure the **Maps Tile API** and **Places API (new)** are enabled for this key in your GCP console.

2.  **Run the Cells:** Once the secrets are set, simply run the cells in order from top to bottom. Each visualization will appear as the output of its corresponding code cell.

In [ ]:
!pip install google-maps-places --quiet

In [ ]:
# Import libraries
from google.colab import auth, userdata, data_table

from google.maps import places_v1
import asyncio

import requests

import geopandas as gpd
import shapely
import sys

import pandas as pd

# Import the mapping libraries
import folium
import mapclassify # Used by .explore() for data classification
import xyzservices # Provides tile layers

In [ ]:
# Configure GCP Authentication
# This part securely gets your GCP Project ID.
GCP_PROJECT_SECRET_KEY_NAME = "GCP_PROJECT" #@param {type:"string"}
GCP_PROJECT_ID = None

if "google.colab" in sys.modules:
    try:
        GCP_PROJECT_ID = userdata.get(GCP_PROJECT_SECRET_KEY_NAME)
        if GCP_PROJECT_ID:
            print(f"Authenticating to GCP project: {GCP_PROJECT_ID}")
            auth.authenticate_user(project_id=GCP_PROJECT_ID)
        else:
            raise ValueError(f"Could not retrieve GCP Project ID from secret named '{GCP_PROJECT_SECRET_KEY_NAME}'. "
                             "Please make sure the secret is set in your Colab environment.")
    except userdata.SecretNotFoundError:
        raise ValueError(f"Secret named '{GCP_PROJECT_SECRET_KEY_NAME}' not found. "
                         "Please create it in the 'Secrets' tab (key icon) in Colab.")

In [ ]:
API_KEY_SECRET_NAME = "GMP_API_KEY" #@param {type:"string"}

# Initialize a variable to hold our key.
gmp_api_key = None

try:
  # Attempt to retrieve the secret value using its name.
  gmp_api_key = userdata.get(API_KEY_SECRET_NAME)
  print("Successfully retrieved API key.")

except userdata.SecretNotFoundError:
  raise ValueError(f"Secret named '{API_KEY_SECRET_NAME}' not found. "
                         "Please create it in the 'Secrets' tab (key icon) in Colab.")

### **Visualizing Competitor Density on a Heatmap**

The code below performs a high-level competitive analysis by querying for specific competitor brands across Clark County. It then generates a heatmap to visualize areas of market saturation. Here's what it shows:

*   **Weighted Heatmap:** The map displays heat based on the concentration of major competitor brands (`7-Eleven`, `CVS`, `Walgreens`, etc.). The intensity of the "heat" is centered on the centroid of each ZIP code.
*   **Color Scale:** The map uses a gradient from blue (low density) to red (high density), making it easy to spot competitive hotspots at a glance.
*   **Geographic Scope:** The analysis is focused on Clark County, Nevada, providing a broad overview of the Las Vegas metropolitan area.

This visualization gives an immediate, high-level understanding of which ZIP codes are already heavily saturated with key competitors.

**Note:** Upcoming cells uses 2D Map Tiles. Please review the documentation for pricing.

In [ ]:
%%bigquery gdf_zip_counts --project $GCP_PROJECT_ID
WITH brand_counts_by_zip AS (
  SELECT WITH AGGREGATION_THRESHOLD
    postal_code,
    COUNT(*) AS total_brand_count
  FROM
    `places_insights___us.places` AS places_table,
    UNNEST(places_table.postal_code_names) AS postal_code,
    UNNEST(places_table.brand_ids) AS brand_id
  JOIN
    (
      SELECT id, name
      FROM `places_insights___us.brands`
      WHERE name IN ('7-Eleven', 'CVS', 'Walgreens', 'Subway Restaurants', "McDonald's")
    ) AS brand_names
    ON brand_names.id = brand_id
  WHERE
    places_table.administrative_area_level_2_name = 'Clark County'
    AND places_table.administrative_area_level_1_name = 'Nevada'
  GROUP BY postal_code
)
SELECT
  counts.postal_code,
  counts.total_brand_count,
  -- We wrap the geometry in ST_ASTEXT to ensure it downloads as a string
  ST_ASTEXT(ST_SIMPLIFY(zip_boundaries.zip_code_geom, 100)) AS geography
FROM
  brand_counts_by_zip AS counts
JOIN
  `bigquery-public-data.geo_us_boundaries.zip_codes` AS zip_boundaries
  ON counts.postal_code = zip_boundaries.zip_code
ORDER BY
  counts.total_brand_count DESC

In [ ]:
from shapely import wkt # type: ignore
import geopandas as gpd

# 1. Convert the WKT string column into actual geometry objects
gdf_zip_counts['geometry'] = gdf_zip_counts['geography'].apply(wkt.loads) # type: ignore

# 2. Convert the standard DataFrame to a GeoDataFrame
gdf_zip_counts = gpd.GeoDataFrame(gdf_zip_counts, geometry='geometry', crs="EPSG:4326") # type: ignore

print("Conversion successful. Displaying top 5 ZIP codes:")
display(gdf_zip_counts.head(5))

In [ ]:
# Import the HeatMap plugin from folium
from folium.plugins import HeatMap

# Verify the GMP API key exists.
if 'gmp_api_key' not in locals() or gmp_api_key is None:
    raise NameError("The 'gmp_api_key' variable is not defined or is None. "
                    "Please run the API key retrieval cell first.")

# Request a session token and attribution for Google Maps tiles.
print("Requesting Google Maps session token...")
session_url = f"https://tile.googleapis.com/v1/createSession?key={gmp_api_key}"
payload = {"mapType": "roadmap", "language": "en-US", "region": "US"}
headers = {"Content-Type": "application/json"}

try:
    response_session = requests.post(session_url, json=payload, headers=headers)
    response_session.raise_for_status() # Raise an error for bad responses
    session_data = response_session.json()
    session_token = session_data['session']
    print("Session token acquired successfully.")

    # Fetch dynamic attribution required by Google Maps.
    print("Fetching dynamic attribution...")
    bounds = gdf_zip_counts.total_bounds
    viewport_url = (
        f"https://tile.googleapis.com/tile/v1/viewport?key={gmp_api_key}"
        f"&session={session_token}"
        f"&zoom=10"
        f"&north={bounds[3]}&south={bounds[1]}"
        f"&west={bounds[0]}&east={bounds[2]}"
    )
    response_viewport = requests.get(viewport_url)
    response_viewport.raise_for_status()
    viewport_data = response_viewport.json()
    google_attribution = viewport_data.get('copyright', '© Google')
    print("Attribution received.")

except requests.exceptions.RequestException as e:
    raise RuntimeError(f"Failed to set up Google Maps tiles. Please check your API key and permissions. Details: {e}")

# Construct the Tile URL that Folium will use for the basemap.
google_tiles_url = f"https://tile.googleapis.com/v1/2dtiles/{{z}}/{{x}}/{{y}}?session={session_token}&key={gmp_api_key}"

# Ensure the geometry column is correctly named 'geometry' for GeoPandas operations.
if 'geography' in gdf_zip_counts.columns and 'geometry' not in gdf_zip_counts.columns:
    gdf_zip_counts = gdf_zip_counts.rename(columns={'geography': 'geometry'})
    # Also, explicitly set it as the active geometry column.
    gdf_zip_counts = gdf_zip_counts.set_geometry('geometry')

# Calculate the center of the data to focus the map.
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2
map_center = [center_lat, center_lon]

# Initialize the base map.
print("Initializing base map...")
competitor_heatmap = folium.Map(
    location=map_center,
    zoom_start=10,
    tiles=google_tiles_url,
    attr=google_attribution
)

# Prepare data for the heatmap: a list of [lat, lon, weight] points.
print("Preparing weighted locations from ZIP code centroids...")
weighted_locations = []
for index, row in gdf_zip_counts.iterrows():
    # Get the center point (centroid) of each ZIP code polygon.
    # This line will now work correctly.
    centroid = row['geometry'].centroid
    # Append the centroid's lat/lon and the brand count as its weight.
    weighted_locations.append([centroid.y, centroid.x, row['total_brand_count']])

# Create and add the heatmap layer to the base map.
print("Generating and adding heatmap layer...")
heatmap_layer = HeatMap(
    data=weighted_locations,
    radius=25, # Adjust the influence radius of each data point
    blur=15    # Adjust the smoothness of the color gradient
)
heatmap_layer.add_to(competitor_heatmap)

# Display the final map.
print("Displaying map...")
display(competitor_heatmap)

### **Mapping Commercial Suitability with a Choropleth Map**

The code below executes a sophisticated query to calculate a custom `suitability_score` for hexagonal H3 cells across Las Vegas. Here's what the resulting map shows:

*   **Choropleth Map:** Each H3 hexagon on the map is colored based on its normalized suitability score, which is calculated from the weighted density of nearby complementary businesses (restaurants, bars, casinos, etc.).
*   **Color Scale:** The map uses a light-to-dark sequential purple scale (`Purples`). Lighter purple areas have lower suitability scores, while the darkest purple areas represent the most commercially vibrant hotspots.
*   **Interactivity:** You can hover over any hexagon to see its unique H3 index, its suitability score, and the specific counts of businesses that contributed to that score. Clicking on a hexagon will open a popup with all the data for that cell.

This visualization moves beyond simple competitor analysis to pinpoint the most promising micro-locations for a new business based on commercial synergy.

**Note:** Upcoming cells uses 2D Map Tiles. Please review the documentation for pricing.

In [ ]:
%%bigquery gdf_suitability --project $GCP_PROJECT_ID
WITH PlacesInTargetAreaWithOpenFlag AS (
  SELECT
    point,
    types,
    EXISTS(
      SELECT 1
      FROM UNNEST(regular_opening_hours.monday) AS monday_hours
      WHERE
        monday_hours.start_time <= TIME '10:00:00'
        AND monday_hours.end_time >= TIME '14:00:00'
    ) AS is_open_monday_window
  FROM
    `places_insights___us.places`
  WHERE
    EXISTS (
        SELECT 1 FROM UNNEST(locality_names) AS locality
        WHERE locality IN ('Las Vegas', 'Spring Valley', 'Paradise', 'North Las Vegas', 'Winchester')
    )
    AND administrative_area_level_1_name = 'Nevada'
),
TileScores AS (
  SELECT WITH AGGREGATION_THRESHOLD
    `carto-os.carto.H3_FROMGEOGPOINT`(point, 8) AS h3_index,
    (
      COUNTIF('restaurant' IN UNNEST(types) AND is_open_monday_window) * 8 +
      COUNTIF('convenience_store' IN UNNEST(types) AND is_open_monday_window) * 3 +
      COUNTIF('bar' IN UNNEST(types) AND is_open_monday_window) * 7 +
      COUNTIF('tourist_attraction' IN UNNEST(types) AND is_open_monday_window) * 6 +
      COUNTIF('casino' IN UNNEST(types) AND is_open_monday_window) * 7
    ) AS suitability_score,
    COUNTIF('restaurant' IN UNNEST(types) AND is_open_monday_window) AS restaurant_count,
    COUNTIF('convenience_store' IN UNNEST(types) AND is_open_monday_window) AS convenience_store_count,
    COUNTIF('bar' IN UNNEST(types) AND is_open_monday_window) AS bar_count,
    COUNTIF('tourist_attraction' IN UNNEST(types) AND is_open_monday_window) AS tourist_attraction_count,
    COUNTIF('casino' IN UNNEST(types) AND is_open_monday_window) AS casino_count
  FROM
    PlacesInTargetAreaWithOpenFlag
  GROUP BY
    h3_index
),
MaxScore AS (
  SELECT MAX(suitability_score) AS max_score FROM TileScores
)
SELECT
  ts.h3_index,
  -- We calculate the H3 Boundary and convert to Text for safe transport
  ST_ASTEXT(`carto-os.carto.H3_BOUNDARY`(ts.h3_index)) AS geometry,
  ts.restaurant_count,
  ts.convenience_store_count,
  ts.bar_count,
  ts.tourist_attraction_count,
  ts.casino_count,
  ts.suitability_score,
  ROUND(
    CASE
      WHEN ms.max_score = 0 THEN 0
      ELSE (ts.suitability_score / ms.max_score) * 10
    END,
    2
  ) AS normalized_suitability_score
FROM
  TileScores ts, MaxScore ms
ORDER BY
  normalized_suitability_score DESC;

In [ ]:
from shapely import wkt # type: ignore
import geopandas as gpd

# 1. Convert the WKT string column into actual geometry objects
gdf_suitability['geometry'] = gdf_suitability['geometry'].apply(wkt.loads) # type: ignore

# 2. Convert to GeoDataFrame
gdf_suitability = gpd.GeoDataFrame(gdf_suitability, geometry='geometry', crs="EPSG:4326") # type: ignore

print("Suitability scores loaded successfully. Displaying top 5 H3 cells:")
display(gdf_suitability.head(5))

In [ ]:
try:
    # This code block assumes 'google_tiles_url' and 'google_attribution'
    # were successfully created and are available from a previous cell.
    _ = google_tiles_url
    _ = google_attribution
except NameError:
    raise NameError("The required variables 'google_tiles_url' or 'google_attribution' were not found. "
                    "Please run the previous map-generating cell to create them before running this one.")


# Define the columns to show in the map's tooltip on hover.
suitability_tooltip_cols = [
    'h3_index',
    'normalized_suitability_score',
    'suitability_score',
    'restaurant_count',
    'casino_count',
    'bar_count',
    'tourist_attraction_count',
    'convenience_store_count'
]

# Create the choropleth map using the .explore() function.
print("Generating choropleth map of suitability scores using a purple color scheme...")
suitability_map = gdf_suitability.explore(
    column="normalized_suitability_score",
    # *** CHANGE APPLIED HERE ***
    # The 'Purples' colormap provides a sequential light-to-dark purple scale.
    cmap="Purples",
    scheme="NaturalBreaks",
    k=7,
    tooltip=suitability_tooltip_cols,
    popup=True,
    tiles=google_tiles_url, # Reusing the variable from the previous cell
    attr=google_attribution,  # Reusing the variable from the previous cell
    style_kwds={"stroke": True, "color": "black", "weight": 0.2, "fillOpacity": 0.65}
)

# Display the final map.
print("Displaying map...")
display(suitability_map)

### Calculating Coffee Shop Density using a Specialized Function**

The code below uses the `PLACES_COUNT_PER_H3` function in Place Insights to efficiently count existing coffee shops within the Las Vegas metro area. Here's what the query does:

*   **Geofenced Query:** The analysis is automatically constrained to a predefined Las Vegas metro area, created by merging the boundaries of five key localities.
*   **H3 Aggregation:** The function returns precise coffee shop counts neatly aggregated into H3 resolution 8 cells.
*   **Data Output:** The result is a GeoDataFrame containing the H3 index, the total coffee shop count for that cell, and a list of `sample_place_ids` for spot-checking.

This step provides the crucial, quantitative data on the density of our target business type in preparation for the final analysis.

In [ ]:
%%bigquery gdf_coffee_shops --project $GCP_PROJECT_ID
-- Define a variable to hold the combined geography for the Las Vegas metro area.
DECLARE las_vegas_metro_area GEOGRAPHY;

-- Set the variable by fetching the shapes for the five localities
SET las_vegas_metro_area = (
  SELECT
    ST_UNION_AGG(geometry)
  FROM
    `bigquery-public-data.overture_maps.division_area`
  WHERE
    country = 'US'
    AND region = 'US-NV'
    AND names.primary IN ('Las Vegas', 'Spring Valley', 'Paradise', 'North Las Vegas', 'Winchester')
);

-- Call the function and select results
SELECT
  h3_cell_index,
  count,
  sample_place_ids,
  -- FIX: The function returns a column named 'geography', not 'h3_geography'
  ST_ASTEXT(geography) AS geometry
FROM
  `places_insights___us.PLACES_COUNT_PER_H3`(
    JSON_OBJECT(
      'geography', las_vegas_metro_area,
      'types', ["coffee_shop"],
      'business_status', ['OPERATIONAL'],
      'h3_resolution', 8
    )
  );

In [ ]:
from shapely import wkt # type: ignore
import geopandas as gpd

# 1. Convert the WKT string column into actual geometry objects
gdf_coffee_shops['geometry'] = gdf_coffee_shops['geometry'].apply(wkt.loads) # type: ignore

# 2. Convert to GeoDataFrame
gdf_coffee_shops = gpd.GeoDataFrame(gdf_coffee_shops, geometry='geometry', crs="EPSG:4326") # type: ignore

print("Coffee shop counts loaded successfully. Displaying top 5 H3 cells:")
display(gdf_coffee_shops.sort_values('count', ascending=False).head(5))

In [ ]:
# This cell performs a comparative analysis. It identifies the top 5 zones from your
# custom suitability model and then cross-references them with the coffee shop
# density data to see how they compare.

try:
    # --- Step 1: Isolate the Top 5 Most Suitable H3 Cells ---
    # The 'gdf_suitability' DataFrame is already sorted by score, so we just
    # need to take the first 5 rows.
    print("Identifying the top 5 H3 cells from the suitability score analysis...")
    top_suitability_cells = gdf_suitability.head(5)

    # Extract the 'h3_index' values from these top 5 cells into a list.
    top_h3_indexes = top_suitability_cells['h3_index'].tolist()
    print(f"The top 5 H3 indexes are: {top_h3_indexes}")

    # Now, we find the rows in our 'gdf_coffee_shops' DataFrame where the
    # 'h3_cell_index' matches one of the indexes from our top 5 list.

    # The .isin() method is perfect for filtering a DataFrame based on a list of values.
    # Note: The column name in this DataFrame is 'h3_cell_index'.
    coffee_counts_in_top_zones = gdf_coffee_shops[
        gdf_coffee_shops['h3_cell_index'].isin(top_h3_indexes)
    ]

    # --- Step 3: Display the Final Comparison Table ---
    print("\n--- Results ---")
    print("Coffee shop counts within the top 5 most suitable zones:")

    if coffee_counts_in_top_zones.empty:
        print("No coffee shops were found in the top 5 most suitable H3 cells.")
    else:
        # Display the resulting table, sorted by coffee shop count for clarity.
        display(coffee_counts_in_top_zones.sort_values('count', ascending=False))

except NameError as e:
    print(f"\nERROR: A required DataFrame was not found. Please ensure the cells that create "
          f"'gdf_suitability' and 'gdf_coffee_shops' have been run successfully.")
    print(f"Details: {e}")
except KeyError as e:
    print(f"\nERROR: A required column was not found. Please check the column names in your DataFrames.")
    print(f"Details: {e}")

### **Fetching Ground-Truth Details with the Places API**

The code below uses the `sample_place_ids` from our highest-potential zones to fetch detailed, ground-truth information for each coffee shop using the Places API. Here's what it does:

*   **Places API Call:** The code programmatically calls the Places API for each unique Place ID, requesting specific fields to optimize for cost and speed.
*   **Rich Information:** It retrieves key details like the business's official name, formatted address, user rating, website, and precise latitude/longitude coordinates.
*   **Structured Output:** The results are compiled into a clean and readable Pandas DataFrame.

This step transforms the aggregated counts from the previous analysis into a concrete list of real-world competitor locations.

**Note:** This cell uses the Places API. Please review the documentation for pricing.

In [ ]:

#import pandas as pd

# Verify that the required DataFrame from the previous step exists.
if 'coffee_counts_in_top_zones' not in locals():
    raise NameError("The 'coffee_counts_in_top_zones' DataFrame was not found. "
                    "Please run the previous cell to create it before proceeding.")

# --- Step 1: Extract and Unify All Place IDs ---
# The 'sample_place_ids' column contains lists of IDs. We need to flatten this
# into a single list and get only the unique IDs to avoid duplicate API calls.
print("Extracting unique Place IDs from the data...")
all_place_ids = [
    place_id
    for id_list in coffee_counts_in_top_zones['sample_place_ids']
    for place_id in id_list
]
unique_place_ids = list(set(all_place_ids))
print(f"Found {len(unique_place_ids)} unique places to look up.")

# --- Step 2: Initialize the Places API Client ---
# We use the GMP API key that was securely stored in Colab's userdata.
print("Initializing Places API client...")
try:
    places_client = places_v1.PlacesAsyncClient(
        client_options={"api_key": gmp_api_key}
    )
except NameError:
    raise NameError("The 'gmp_api_key' variable is not defined. Please run the API key cell first.")

# --- Step 3: Define an Asynchronous Function to Fetch Details ---
# Using an async function allows us to make API calls concurrently in the future if needed.
async def get_details_for_places(client, place_ids):
    """Fetches details for a list of Place IDs using the Places API."""
    place_details_list = []
    print(f"Fetching details for {len(place_ids)} places...")

    for place_id in place_ids:
        # The 'name' parameter for the API must be in the format 'places/PLACE_ID'.
        request = places_v1.GetPlaceRequest(
            name=f"places/{place_id}",
        )

        # A FieldMask specifies which fields we want in the response.
        # This is a best practice that reduces latency and cost.
        # *** CHANGE APPLIED HERE: Added 'location' to the field mask ***
        field_mask = "displayName,formattedAddress,rating,websiteUri,location"

        try:
            # Make the asynchronous API call.
            response = await client.get_place(request=request, metadata=[('x-goog-fieldmask', field_mask)])

            # Store the results in a dictionary.
            # *** CHANGE APPLIED HERE: Added latitude and longitude from the response ***
            details = {
                "place_id": place_id,
                "name": response.display_name.text,
                "address": response.formatted_address,
                "rating": response.rating,
                "website": response.website_uri,
                "latitude": response.location.latitude,
                "longitude": response.location.longitude
            }
            place_details_list.append(details)

        except Exception as e:
            print(f"  - Could not retrieve details for Place ID {place_id}: {e}")

    return place_details_list

# --- Step 4: Run the Function and Display the Results ---
# In a Colab notebook, we can directly 'await' our async function.
place_details_results = await get_details_for_places(places_client, unique_place_ids)

# Convert the list of dictionaries into a Pandas DataFrame for clean display.
if place_details_results:
    details_df = pd.DataFrame(place_details_results)
    print("\n--- Place Details Results ---")
    display(details_df)
else:
    print("\nNo details were successfully retrieved.")

### **Creating a Combined Visualization for Final Analysis**

This final cell synthesizes all our previous findings onto a single, layered, and interactive map for a comprehensive strategic overview. Here’s what it shows:

*   **Base Layer (Choropleth):** The map's foundation is our purple suitability choropleth, clearly highlighting the most commercially active zones in the city.
*   **Top Layer (Markers):** Individual coffee cup icons are overlaid on the map, pinpointing the exact locations of existing coffee shops within our top-ranked suitability zones.
*   **Interactivity:** You can explore the map by hovering over hexagons to see suitability data or by clicking on the coffee cup markers to see specific details for each shop, including its name, address, and rating.

This layered map provides a powerful decision-making tool, allowing you to visually correlate high-potential areas with the current competitive landscape on the ground.

**Note:** This cell uses 2D Map Tiles. Please review the documentation for pricing.

In [ ]:
# import folium

# --- WARNING: This cell reuses variables from a previous cell. ---
# Re-running the cell that creates 'google_tiles_url' is recommended for a fresh session token.

try:
    # Verify all required variables from previous steps exist.
    _ = google_tiles_url
    _ = google_attribution
    _ = gdf_suitability
    _ = details_df
except NameError as e:
    raise NameError(f"A required variable or DataFrame was not found. Please ensure all previous cells "
                    f"have been run successfully. Missing: {e}")

# --- Step 1: Create the Base Choropleth Map ---
# This is the same code as before, creating our purple suitability map.
# The map object is stored in the 'combined_map' variable.
print("Generating the base choropleth map of suitability scores...")

suitability_tooltip_cols = [
    'h3_index', 'normalized_suitability_score', 'suitability_score',
    'restaurant_count', 'casino_count', 'bar_count',
    'tourist_attraction_count', 'convenience_store_count'
]

combined_map = gdf_suitability.explore(
    column="normalized_suitability_score",
    cmap="Purples",
    scheme="NaturalBreaks",
    k=7,
    tooltip=suitability_tooltip_cols,
    popup=True,
    tiles=google_tiles_url,
    attr=google_attribution,
    style_kwds={"stroke": True, "color": "black", "weight": 0.2, "fillOpacity": 0.65}
)

# --- Step 2: Add Markers for Individual Coffee Shops ---
# Now, we loop through our DataFrame of coffee shop details and add a pin for each.
print(f"Adding {len(details_df)} coffee shop markers to the map...")

for index, row in details_df.iterrows():
    # Format the popup content with HTML for better readability.
    popup_html = f"""
    <b>{row['name']}</b><br>
    Address: {row['address']}<br>
    Rating: {row['rating']}
    """

    # Create a Marker for the current coffee shop.
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        # The popup appears when you click the marker.
        popup=folium.Popup(popup_html, max_width=300),
        # The tooltip appears when you hover over the marker.
        tooltip=row['name'],
        # Use a custom icon to clearly represent a coffee shop.
        icon=folium.Icon(color='green', icon='coffee', prefix='fa')
    ).add_to(combined_map) # Add the marker to our map object.


# --- Step 3: Display the Final Combined Map ---
print("Displaying combined map...")
display(combined_map)